# 2.1 토큰화 코드

In [ ]:
# 띄어쓰기 단위로 분리
input_text = "나는 최근 파리 여행을 다녀왔다"
input_text_list = input_text.split()

print("input_text_list: ", input_text_list)

input_text_list:  ['나는', '최근', '파리', '여행을', '다녀왔다']


In [ ]:
# 토큰 -> 아이디 딕셔너리 만들기
str2idx = {word:idx for idx, word in enumerate(input_text_list)}

print("str2idx: ", str2idx)

str2idx:  {'나는': 0, '최근': 1, '파리': 2, '여행을': 3, '다녀왔다': 4}


In [ ]:
# 아이디 -> 토큰 딕셔너리 만들기
idx2str = {idx:word for idx, word in enumerate(input_text_list)}

print("idx2str: ", idx2str)

idx2str:  {0: '나는', 1: '최근', 2: '파리', 3: '여행을', 4: '다녀왔다'}


In [ ]:
# 토큰을 토큰 아이디로 변환
input_ids = [str2idx[word] for word in input_text_list]

print("input_ids: ", input_ids)

input_ids:  [0, 1, 2, 3, 4]


# 2.2 토큰 아이디에서 벡터로 변환

In [ ]:
import torch
import torch.nn as nn

embedding_dim = 16
embed_layer = nn.Embedding(len(str2idx), embedding_dim)

input_embeddings = embed_layer(torch.tensor(input_ids)) # (5, 16)
input_embeddings = input_embeddings.unsqueeze(0) # (1, 5, 16)

input_embeddings.shape

torch.Size([1, 5, 16])

In [ ]:
input_embeddings[0][0]

tensor([-0.3614, -0.0572, -0.4588,  1.3971,  1.4247,  0.8740, -2.5909, -1.3791,
         0.5677,  1.4021,  0.0903, -2.6209,  1.4019,  0.8354,  0.6523,  0.4604],
       grad_fn=<SelectBackward0>)

# 2.3 절대적 위치 인코딩

In [ ]:
embedding_dim = 16
max_position = 12

# 토큰 임베딩 층 생성
embed_layer = nn.Embedding(len(str2idx), embedding_dim)

# 위치 인코딩 층 생성
position_embed_layer = nn.Embedding(max_position, embedding_dim)
position_ids = torch.arange(len(input_ids), dtype=torch.long).unsqueeze(0)
position_encodings = position_embed_layer(position_ids)

token_embeddings = embed_layer(torch.tensor(input_ids)) # (5, 16)
token_embeddings = token_embeddings.unsqueeze(0) # (1, 5, 16)

# 토큰 임베딩과 위치 인코딩을 더해 최종 입력 임베딩 생성
input_embeddings = token_embeddings + position_encodings

input_embeddings.shape

torch.Size([1, 5, 16])

# 2.4 쿼리, 키, 값 벡터를 만드는 nn.Linear 층

In [ ]:
head_dim = 16

# 쿼리, 키, 값을 계산하기 위한 변환
weight_q = nn.Linear(embedding_dim, head_dim)
weight_k = nn.Linear(embedding_dim, head_dim)
weight_v = nn.Linear(embedding_dim, head_dim)

# 변환 수행
querys = weight_q(input_embeddings) # (1, 5, 16)
keys = weight_k(input_embeddings) # (1, 5, 16)
values = weight_v(input_embeddings) # (1, 5, 16)

# 2.5. 스케일 점곱 방식의 어텐션

In [ ]:
from math import sqrt
import torch.nn.functional as F

def compute_attention(querys, keys, values, is_causal=False):
	dim_k = querys.size(-1) # 16
	scores = querys @ keys.transpose(-2, -1) / sqrt(dim_k)
	weights = F.softmax(scores, dim=-1)
	return weights @ values

# 2.6. 어텐션 연산의 입력과 출력

In [ ]:
print("원본 입력 형태: ", input_embeddings.shape)

after_attention_embeddings = compute_attention(querys, keys, values)

print("어텐션 적용 후 형태: ", after_attention_embeddings.shape)
# 원본 입력 형태:  torch.Size([1, 5, 16])
# 어텐션 적용 후 형태:  torch.Size([1, 5, 16])

원본 입력 형태:  torch.Size([1, 5, 16])
어텐션 적용 후 형태:  torch.Size([1, 5, 16])


# 2.7. 어텐션 연산을 수행하는 AttentionHead 클래스

In [ ]:
class AttentionHead(nn.Module):
  def __init__(self, token_embed_dim, head_dim, is_causal=False):
    super().__init__()
    self.is_causal = is_causal
    self.weight_q = nn.Linear(token_embed_dim, head_dim) # 쿼리 벡터 생성을 위한 선형 층
    self.weight_k = nn.Linear(token_embed_dim, head_dim) # 키 벡터 생성을 위한 선형 층
    self.weight_v = nn.Linear(token_embed_dim, head_dim) # 값 벡터 생성을 위한 선형 층

  def forward(self, querys, keys, values):
    outputs = compute_attention(
        self.weight_q(querys),  # 쿼리 벡터
        self.weight_k(keys),    # 키 벡터
        self.weight_v(values),  # 값 벡터
        is_causal=self.is_causal
    )
    return outputs

attention_head = AttentionHead(embedding_dim, embedding_dim)
after_attention_embeddings = attention_head(input_embeddings, input_embeddings, input_embeddings)

after_attention_embeddings

tensor([[[ 0.4047,  0.0939,  0.2441, -0.6583, -0.0134, -0.5776,  0.1498,
           0.0910, -0.4203, -0.4312, -0.8504,  0.0919,  0.8008,  0.4840,
           0.1754,  0.7259],
         [-0.1425,  0.0932,  0.0728, -0.5479,  0.1635, -0.4232,  0.2481,
           0.4192, -0.2860, -0.5058, -0.8056, -0.2665,  1.1062,  0.3470,
           0.2251,  0.7241],
         [-0.0825, -0.3060, -0.2285, -0.2340, -0.0601, -0.2044,  0.3065,
           0.2013, -0.1683, -0.3365, -0.8423, -0.3528,  1.4507,  0.4539,
           0.4230,  0.7430],
         [-0.1684, -0.1617, -0.2509, -0.2396,  0.1444,  0.0652,  0.1169,
           0.2352, -0.1266, -0.4174, -0.4947, -0.3010,  1.3712,  0.2729,
           0.3789,  0.5575],
         [-0.2229,  0.3545,  0.2189, -0.7134,  0.3586, -0.4422,  0.1550,
           0.5587, -0.3308, -0.6263, -0.6667, -0.2160,  0.9078,  0.2232,
           0.1073,  0.6517]]], grad_fn=<UnsafeViewBackward0>)

# 2.8. 멀티 헤드 어텐션 구현

In [ ]:
class MultiheadAttention(nn.Module):
  def __init__(self, token_embed_dim, d_model, n_head, is_causal=False):
    super().__init__()
    self.n_head = n_head
    self.is_causal = is_causal
    self.weight_q = nn.Linear(token_embed_dim, d_model)
    self.weight_k = nn.Linear(token_embed_dim, d_model)
    self.weight_v = nn.Linear(token_embed_dim, d_model)
    self.concat_linear = nn.Linear(d_model, d_model)

  def forward(self, querys, keys, values):
    B, T, C = querys.size()
    querys = self.weight_q(querys).view(B, T, self.n_head, C // self.n_head).transpose(1, 2)
    keys = self.weight_k(keys).view(B, T, self.n_head, C // self.n_head).transpose(1, 2)
    values = self.weight_v(values).view(B, T, self.n_head, C // self.n_head).transpose(1, 2)
    attention = compute_attention(querys, keys, values, self.is_causal)
    output = attention.transpose(1, 2).contiguous().view(B, T, C)
    output = self.concat_linear(output)
    return output

n_head = 4
mh_attention = MultiheadAttention(embedding_dim, embedding_dim, n_head)
after_attention_embeddings = mh_attention(input_embeddings, input_embeddings, input_embeddings)

after_attention_embeddings.shape

torch.Size([1, 5, 16])

# 2.9. 층 정규화 코드

In [ ]:
norm = nn.LayerNorm(embedding_dim)
norm_x = norm(input_embeddings)
norm_x.shape # torch.Size([1, 5, 16])

norm_x.mean(dim=-1).data, norm_x.std(dim=-1).data

# (tensor([[ 2.2352e-08, -1.1176e-08, -7.4506e-09, -3.9116e-08, -1.8626e-08]]),
#  tensor([[1.0328, 1.0328, 1.0328, 1.0328, 1.0328]]))

(tensor([[-3.7253e-08,  3.7253e-08,  1.4901e-08,  3.7253e-08, -3.1665e-08]]),
 tensor([[1.0328, 1.0328, 1.0328, 1.0328, 1.0328]]))

In [ ]:
input_embeddings[0][0]

tensor([ 1.6355, -0.0369,  1.4637,  1.9565,  0.4546, -0.8515,  1.0669,  4.1970,
         0.1689, -0.2110,  1.9591,  0.3783, -0.1490, -3.9026,  2.3559,  0.3503],
       grad_fn=<SelectBackward0>)

In [ ]:
input_embeddings[0][0].mean(), input_embeddings[0][0].std()

(tensor(0.6772, grad_fn=<MeanBackward0>),
 tensor(1.7507, grad_fn=<StdBackward0>))

In [ ]:
norm_x[0][0]

tensor([ 0.5653, -0.4213,  0.4640,  0.7547, -0.1314, -0.9019,  0.2299,  2.0765,
        -0.2999, -0.5240,  0.7562, -0.1764, -0.4874, -2.7018,  0.9903, -0.1929],
       grad_fn=<SelectBackward0>)

In [ ]:
norm_x[0][0].mean(), norm_x[0][0].std()

(tensor(-3.7253e-08, grad_fn=<MeanBackward0>),
 tensor(1.0328, grad_fn=<StdBackward0>))

# 2.10. 피드 포워드 층 코드

In [ ]:
class PreLayerNormFeedForward(nn.Module):
  def __init__(self, d_model, dim_feedforward, dropout):
    super().__init__()
    self.linear1 = nn.Linear(d_model, dim_feedforward) # 선형 층 1
    self.linear2 = nn.Linear(dim_feedforward, d_model) # 선형 층 2
    self.dropout1 = nn.Dropout(dropout) # 드랍아웃 층 1
    self.dropout2 = nn.Dropout(dropout) # 드랍아웃 층 2
    self.activation = nn.GELU() # 활성 함수
    self.norm = nn.LayerNorm(d_model) # 층 정규화

  def forward(self, src):
    x = self.norm(src)
    x = x + self.linear2(self.dropout1(self.activation(self.linear1(x))))
    x = self.dropout2(x)
    return x

# 2.11. 인코더 층

In [ ]:
class TransformerEncoderLayer(nn.Module):
  def __init__(self, d_model, nhead, dim_feedforward, dropout):
    super().__init__()
    self.attn = MultiheadAttention(d_model, d_model, nhead) # 멀티 헤드 어텐션 클래스
    self.norm1 = nn.LayerNorm(d_model) # 층 정규화
    self.dropout1 = nn.Dropout(dropout) # 드랍아웃
    self.feed_forward = PreLayerNormFeedForward(d_model, dim_feedforward, dropout) # 피드포워드

  def forward(self, src):
    norm_x = self.norm1(src)
    attn_output = self.attn(norm_x, norm_x, norm_x)
    x = src + self.dropout1(attn_output) # 잔차 연결

    # 피드 포워드
    x = self.feed_forward(x)
    return x

# 2.12. 인코더 구현

In [ ]:
import copy
def get_clones(module, N):
  return nn.ModuleList([copy.deepcopy(module) for i in range(N)])

class TransformerEncoder(nn.Module):
  def __init__(self, encoder_layer, num_layers):
    super().__init__()
    self.layers = get_clones(encoder_layer, num_layers)
    self.num_layers = num_layers
    self.norm = norm

  def forward(self, src):
    output = src
    for mod in self.layers:
        output = mod(output)
    return output

# 2.13. 디코더에서 어텐션 연산(마스크 어텐션)

In [ ]:
def compute_attention(querys, keys, values, is_causal=False):
  dim_k = querys.size(-1) # 16
  scores = querys @ keys.transpose(-2, -1) / sqrt(dim_k) # (1, 5, 5)

  if is_causal:
    query_length = querys.size(2)
    key_length = keys.size(2)
    temp_mask = torch.ones(query_length, key_length, dtype=torch.bool).tril(diagonal=0)
    scores = scores.masked_fill(temp_mask == False, float("-inf"))

  weights = F.softmax(scores, dim=-1) # (1, 5, 5)
  return weights @ values # (1, 5, 16)

# 2.14. 크로스 어텐션이 포함된 디코더 층

In [ ]:
class TransformerDecoderLayer(nn.Module):
  def __init__(self, d_model, nhead, dim_feedforward=2048, dropout=0.1):
    super().__init__()
    self.self_attn = MultiheadAttention(d_model, d_model, nhead)
    self.multihead_attn = MultiheadAttention(d_model, d_model, nhead)
    self.feed_forward = PreLayerNormFeedForward(d_model, dim_feedforward, dropout)

    self.norm1 = nn.LayerNorm(d_model)
    self.norm2 = nn.LayerNorm(d_model)
    self.dropout1 = nn.Dropout(dropout)
    self.dropout2 = nn.Dropout(dropout)

  def forward(self, tgt, encoder_output, is_causal=True):
    # 셀프 어텐션 연산
    x = self.norm1(tgt)
    x = x + self.dropout1(self.self_attn(x, x, x, is_causal=is_causal))
    # 크로스 어텐션 연산
    x = self.norm2(x)
    x = x + self.dropout2(self.multihead_attn(x, encoder_output, encoder_output))
    # 피드 포워드 연산
    x = self.feed_forward(x)
    return x

# 2.15. 디코더 구현

In [ ]:
import copy
def get_clones(module, N):
  return nn.ModuleList([copy.deepcopy(module) for i in range(N)])

class TransformerDecoder(nn.Module):
  def __init__(self, decoder_layer, num_layers):
    super().__init__()
    self.layers = get_clones(decoder_layer, num_layers)
    self.num_layers = num_layers

  def forward(self, tgt, src):
    output = tgt
    for mod in self.layers:
        output = mod(tgt, src)
    return output